In [23]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer,StandardScaler
from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error

In [24]:
df=pd.read_csv('Encoded_House.csv')

x=df.drop(['log_price','price'],axis=1).values
y=df['log_price'].values.reshape(-1, 1)

In [25]:
#====== Test Train Split======
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#====== Standerization ======
scaler_x = StandardScaler()
x_train = scaler_x.fit_transform(x_train)
x_test = scaler_x.transform(x_test)

#====== Adding bias ======
x_train_bias=np.c_[x_train, np.ones((x_train.shape[0]))]
x_test_bias=np.c_[x_test, np.ones((x_test.shape[0]))]

### OLS Method
β = (XᵀX)⁻¹Xᵀy , To minimize error, we differentiate Error = ||Xβ-Y||^2 with respect to β, and make it equal to 0, thus get β

If two or more columns in X_train_bias are linearly dependent, then XᵀX become singular, Use the Moore-Penrose pseudo-inverse

In [26]:
A=np.linalg.pinv(np.dot(x_train_bias.T,x_train_bias)) 
B=np.dot(x_train_bias.T,y_train)
theta_ols=np.dot(A,B)

#=== Prediction ===
y_pred_ols=np.dot(x_test_bias,theta_ols)

### Gradient Decent Method

In [27]:
def gradient_descent(x, y, lr=0.05, threshold=1e-7, max_iter=100000): 
    m = x.shape[0]
    theta = np.zeros((x.shape[1], 1))
    
    for i in range(max_iter):
        pred = x @ theta
        cost = (1 / (2 * m)) * np.sum((pred - y) ** 2)
        if cost < threshold:
            break

        gradients = (1 / m) * x.T @ (pred - y)
        theta -= lr * gradients

    return theta

#=== Prediction ===
theta_gd = gradient_descent(x_train_bias, y_train, lr=0.05)
y_pred_gd = x_test_bias @ theta_gd

## Ridge Regression

In [28]:
def ridge_regression(x, y, alpha=0.1, lr=0.05, threshold=1e-7, max_iter=100000): 
    m = x.shape[0]
    theta = np.zeros((x.shape[1], 1))
    
    for i in range(max_iter):
        pred = x @ theta
        cost = (1 / (2 * m)) * np.sum((pred - y) ** 2) + (alpha / (2 * m)) * np.sum(theta ** 2)
        if cost < threshold:
            break

        gradients = (1 / m) * x.T @ (pred - y) + (alpha / m) * theta
        theta -= lr * gradients

    return theta

#=== Prediction ===
theta_gd_ridge = ridge_regression(x_train_bias, y_train, lr=0.01)
y_pred_gd_ridge = x_test_bias @ theta_gd_ridge


## Lasso Regression

In [29]:
def lasso_regression(x, y, alpha=0.01, lr=0.05, threshold=1e-7, max_iter=100000): 
    m = x.shape[0]
    theta = np.zeros((x.shape[1], 1))
    
    for i in range(max_iter):
        pred = x @ theta
        cost = (1 / (2 * m)) * np.sum((pred - y) ** 2) + alpha * np.sum(abs(theta))
        if cost < threshold:
            break

        gradients = (1 / m) * x.T @ (pred - y) + alpha*np.sign(theta)
        theta -= lr * gradients

    return theta

#=== Prediction ===
theta_gd_lasso = lasso_regression(x_train_bias, y_train, lr=0.01)
y_pred_gd_lasso = x_test_bias @ theta_gd_lasso

#### Evalution

In [30]:
# ====== Evalution ======
def evaluate(y_true, y_pred, label="Model"):
    r2 = r2_score(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{label} [ R²: {r2:.6f}, RMSE: {rmse:.5f}, MAE: {mae:.5f} ]")

evaluate(y_test, y_pred_ols, "OLS Regression")
evaluate(y_test, y_pred_gd, "Gradient Descent Regression")
evaluate(y_test, y_pred_gd_ridge, "Ridge Regression")
evaluate(y_test, y_pred_gd_lasso, "Lasso Regression")

OLS Regression [ R²: 0.601611, RMSE: 0.45249, MAE: 0.36889 ]
Gradient Descent Regression [ R²: 0.601611, RMSE: 0.45249, MAE: 0.36889 ]
Ridge Regression [ R²: 0.601670, RMSE: 0.45245, MAE: 0.36887 ]
Lasso Regression [ R²: 0.592474, RMSE: 0.45765, MAE: 0.36995 ]
